In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import AlexNet, vgg11_bn, vgg16_bn, resnet18, resnet50, resnet101
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import accuracy_score
import numpy as np

In [10]:
# Hyperparameters
batch_size=64
learning_rate = 0.001
num_epochs = 1

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)
train_dataset = Subset(train_dataset, np.random.choice(len(train_dataset), int(0.25 * len(train_dataset)), replace=False))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [11]:
def compute_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    return accuracy_score(predicted.cpu().numpy(), labels.cpu().numpy())

In [12]:
def train_model(model, criterion, optimizer, num_epochs):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct_predictions += (torch.max(outputs, 1)[1] == labels).sum().item()
            total_predictions += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_predictions

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")


In [15]:
models = {
    'LeNet-5': torchvision.models.googlenet(pretrained=True),
    'VGG11': vgg11_bn(pretrained=True),
    'VGG16': vgg16_bn(pretrained=True),
    'ResNet18': resnet18(pretrained=True),
    'ResNet50': resnet50(pretrained=True),
    'ResNet101': resnet101(pretrained=True),
    'AlexNet': AlexNet()
}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_BN_Weights.DEFAULT` to get the most up-to-date we

In [16]:
# Train models
for name, model in models.items():
    print(f"Training {name}...")

    if 'LeNet' in name:
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
    elif 'AlexNet' in name:
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, 10)
    elif 'VGG' in name:
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, 10)
    elif 'ResNet' in name:
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_model(model, criterion, optimizer, num_epochs)

    model.eval()
    test_accuracy = 0.0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_accuracy += compute_accuracy(outputs, labels) * inputs.size(0)

    test_accuracy /= len(test_loader.dataset)
    print(f"{name} Test Accuracy: {test_accuracy:.4f}")


Training LeNet-5...
Epoch 1/1, Loss: 0.9476, Accuracy: 0.6927
LeNet-5 Test Accuracy: 0.8357
Training VGG11...
Epoch 1/1, Loss: 0.7828, Accuracy: 0.7632


KeyboardInterrupt: 